In [4]:
import pandas as pd
import re
import os
import fitz
from PIL import Image
import pytesseract

In [393]:
pdf_path = 'C3.pdf'

text = ''
pdf_document = fitz.open(pdf_path)
count = pdf_document.page_count
for i in range(count):
    page = pdf_document[i]
    tx= page.get_text()
    text+=tx

In [394]:
# print(text)
k1 = {}

In [395]:
patterns = [
    r'Taxable Sub Total\s+(\d+,\d+\.\d+)',
    # r'IGST\s+\d+% - \d+\%\s+(\d+,\d+\.\d+)',
    r'Non-Taxable Sub Total\s+(\d+,\d+\.\d+)',
    # r'Round Off\s+(\d+\.\d+)',
    r'TOTAL\s+₹\s+(\d+,\d+\.\d+)'
]


results = {}
for pattern in patterns:
    match = re.search(pattern, text)
    if match:
        results[match.group(0)] = match.group(1)

for i in list(results.keys()):
    a,b = i.split('\n')[0],i.split('\n')[1]
    k1[a] = b

In [396]:
k1

{'Taxable Sub Total': '4,075.00', 'TOTAL': '₹ 4,279.00'}

In [397]:
invoice_number_pattern = r'Invoice Number:\s*([A-Z0-9-]+)'
invoice_date_pattern = r'Invoice Date:\s*(\d{2}-\d{2}-\d{4})'

# Search for the patterns in the text
invoice_number_match = re.search(invoice_number_pattern, text)
invoice_date_match = re.search(invoice_date_pattern, text)

# Extract the information if found
invoice_number = invoice_number_match.group(1) if invoice_number_match else None
invoice_date = invoice_date_match.group(1) if invoice_date_match else None

# Print the extracted information
k1["Invoice Number"] =  invoice_number
k1["Invoice Date"] = invoice_date

In [398]:
pan_pattern = r'PAN:\s*([A-Z0-9]+)'
gstin_pattern = r'GSTIN:\s*([A-Z0-9]+)'


pan_match = re.search(pan_pattern, text)
gstin_match = re.search(gstin_pattern, text)


pan_number = pan_match.group(1) if pan_match else None
gstin_number = gstin_match.group(1) if gstin_match else None


k1["PAN"] =  pan_number
k1["GSTIN"] =  gstin_number

In [399]:
k1

{'Taxable Sub Total': '4,075.00',
 'TOTAL': '₹ 4,279.00',
 'Invoice Number': 'AF2223-026870',
 'Invoice Date': '15-03-2023',
 'PAN': 'AAFCI9896L',
 'GSTIN': '24AAFCI9896L1ZQ'}

In [400]:
# pdf_path = 'C1.pdf'

# text = ''
# pdf_document = fitz.open(pdf_path)
# count = pdf_document.page_count
# for i in range(count):
#     if i==0:
#         continue
#     page = pdf_document[i]
#     tx= page.get_text()
#     text+=tx

In [401]:
# print(text)

In [402]:
km_pattern = r'KM\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)'
time_pattern = r'Time\s+(\d{4} \d{2}/\d{2})\s+(\d{4} \d{2}/\d{2})\s+(\d{2}:\d{2})\s+(\d{2}:\d{2})'


km_match = re.search(km_pattern, text)
time_match = re.search(time_pattern, text)


if km_match:
    km_start, km_end, km_total, km_extra = km_match.groups()
else:
    km_start, km_end, km_total, km_extra = None, None, None, None

if time_match:
    time_start, time_end, time_total, time_extra = time_match.groups()
else:
    time_start, time_end, time_total, time_extra = None, None, None, None

k =  {
    "KM Start": km_start,
    "KM End": km_end,
    "KM Total": km_total,
    "KM Extra": km_extra,
    "Time Start": time_start,
    "Time End": time_end,
    "Time Total": time_total,
    "Time Extra": time_extra
}

In [403]:
k

{'KM Start': '134536',
 'KM End': '134601',
 'KM Total': '65',
 'KM Extra': '0',
 'Time Start': '1030 09/02',
 'Time End': '2200 09/02',
 'Time Total': '11:30',
 'Time Extra': '03:30'}

In [408]:
import tabula

# for C3.PDF 
area = [680, 33, 1100, 900]

# for C2.PDF the area will be area = [700, 33, 1100, 900]

def extract_data_from_pdf(pdf_path, area):
    try:
        # Read the PDF using Tabula with the defined area
        tables = tabula.read_pdf(pdf_path, pages='all', area=area)
        return tables
    except Exception as e:
        print(f"Error: {e}")
        return None


pdf_path = "C3.pdf"

# Extract data from PDF using defined area
tables = extract_data_from_pdf(pdf_path, area)
df1 = tables[0]
df1.fillna('', inplace=True)
df1

,Duty Id,Booked\rBy,Passenger,Veh.Group,Veh.Num.,Duty Type,Qty,S.Date,E.Date,S.Time,...,Speedo\rS.Km,Speedo\rE.Km,S.Km,E.Km,T.Km,Ex.Km,Ex.Hr\rRate,Ex.Km\rRate,Ex.Hr\rCost,Ex.Km\rCost
0,#37738732-\r1,Sobhagya\rSingh\rChundawat,"Ravi Kiran\rKolavenne,\rDiwakar\rSharma",INNOVA,MP07 TA\r1629,LOCAL\r8HRS/80KM,1,09-02,09-02,1030,...,134536,134601,134536,134601,65,0,250,21,875,0
1,Total,11:30,,65,,,,,,,...,,,,,,,,,,


In [409]:
ls = list(df1.columns)
k = {}
for i in range(len(ls)):
    k[ls[i].replace('\r',' ') ] = (str(df1[ls[i]].iloc[0]).replace('\r',' '))

In [413]:
import tabula

# Define the coordinates of the area containing the table [top, left, bottom, right]
# area = [480, 20, 1000, 900] for c2

area = [460, 20, 1000, 900] 

# Function to extract data from PDF using Tabula with defined area
def extract_data_from_pdf(pdf_path, area):
    try:
        # Read the PDF using Tabula with the defined area
        tables = tabula.read_pdf(pdf_path, pages='all', area=area)
        return tables
    except Exception as e:
        print(f"Error: {e}")
        return None


pdf_path = "C3.pdf"

# Extract data from PDF using defined area
tables = extract_data_from_pdf(pdf_path, area)
df1 = tables[0]
df1.fillna('', inplace=True)
# df1

In [415]:
Date = df1.loc[0]['Duty #37738732-1'].split(' ')[-1]
driver = df1.loc[3]['Duty #37738732-1'].split(' ')[-1]
reporting_address = (' ').join([s for s in df1.loc[5]['Duty #37738732-1'].split(' ')[2:]])
drop_address = (' ').join([s for s in df1.loc[6]['Duty #37738732-1'].split(' ')[2:]])

In [387]:
# Date,driver,reporting_address,drop_address

In [369]:
# k

In [416]:
k['Date'] = Date
k['driver'] = driver
k['reporting_address'] = reporting_address
k['drop_address'] = drop_address

In [417]:
k.update(k1)

In [418]:
# k

In [419]:
df1 = pd.DataFrame([k])
df1.to_excel('cus_3.xlsx')
df1

,Duty Id,Booked By,Passenger,Veh.Group,Veh.Num.,Duty Type,Qty,S.Date,E.Date,S.Time,...,Date,driver,reporting_address,drop_address,Taxable Sub Total,TOTAL,Invoice Number,Invoice Date,PAN,GSTIN
0,#37738732- 1,Sobhagya Singh Chundawat,"Ravi Kiran Kolavenne, Diwakar Sharma",INNOVA,MP07 TA 1629,LOCAL 8HRS/80KM,1.0,09-02,09-02,1030.0,...,09-02-2023,Yadav,Gwalior Railway Station,Local use,"4,075.00","₹ 4,279.00",AF2223-026870,15-03-2023,AAFCI9896L,24AAFCI9896L1ZQ


In [422]:
df1 = pd.read_excel('cus_1.xlsx',engine = 'openpyxl' )
df2 = pd.read_excel('cus_2.xlsx',engine = 'openpyxl' )
df3 = pd.read_excel('cus_3.xlsx',engine = 'openpyxl' )
main_df = pd.concat([df1, df2,df3], ignore_index=True)

In [426]:
main_df.to_excel('customer.xlsx')